In [1]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [2]:
from wefe.query import Query
from wefe.metrics import WEAT, RNSB, RND, MAC, ECT
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets.datasets import load_weat, fetch_eds, fetch_debias_multiclass, fetch_debiaswe, load_bingliu
from wefe.utils import run_queries, plot_queries_results, create_ranking, plot_ranking, plot_ranking_correlations
from plotly.subplots import make_subplots
import gensim
import numpy as np

In [3]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"

In [4]:
def get_google_news_embeddings(filename,binary=False):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=binary, no_header=True)
    return w2v_model

In [5]:
u_desip_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"debiased_word_embedding/glove_2_wv_U-DeSIP.txt", binary=False)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (322636, 300) matrix of type float32 from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:32:06.628024', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [6]:
sswe_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"/SSWE/embedding-results/sswe-u.bin",binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (137052, 50) matrix of type float32 from ../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-19T13:32:15.447133', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [7]:
p_desip_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"debiased_word_embedding/glove_2_wv_P-DeSIP.txt", binary=False)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (322636, 300) matrix of type float32 from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:32:40.162017', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [8]:
glove_cc_model = get_google_news_embeddings(embeddings_folder+"/Glove/glove.42B.300d.txt")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//Glove/glove.42B.300d.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.42B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.42B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1917494, 300) matrix of type float32 from ../../Data//word embeddings//Glove/glove.42B.300d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:35:00.517952', '

In [9]:
glove_cc_large_model = get_google_news_embeddings(embeddings_folder+"/Glove/glove.840B.300d.txt")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//Glove/glove.840B.300d.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.840B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.840B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2196017, 300) matrix of type float32 from ../../Data//word embeddings//Glove/glove.840B.300d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T13:37:42.049800

#### WEAT Effect Size: represents a normalized measure that quantifies how far apart the two distributions of association between targets and attributes are.

#### WEAT score:  Its objective is to quantify the strength of association of both pair of sets through a permutation test. The score being positive and higher than one indicates that word2vec exhibits a moderately strong relationship between men's names and careers and women's names and family.

In [14]:
# load the weat word sets
WEAT_wordsets = load_weat()
RND_wordsets = fetch_eds()
sentiments_wordsets = load_bingliu()
debias_multiclass_wordsets = fetch_debias_multiclass()

In [10]:
u_desip_model = WordEmbeddingModel(u_desip_model, 'u_desip')
p_desip_model = WordEmbeddingModel(p_desip_model, 'p_desip')
sswe_model = WordEmbeddingModel(sswe_model, 'sswe')
glove_cc_model = WordEmbeddingModel(glove_cc_model, 'glove_cc')
glove_cc_large_model = WordEmbeddingModel(glove_cc_large_model, 'glove_cc_large')


In [11]:
models = [u_desip_model, p_desip_model, sswe_model, glove_cc_model, glove_cc_large_model]

## Query ranking

In [15]:
# Create gender queries
gender_query_1 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']],
                       [WEAT_wordsets['career'], WEAT_wordsets['family']],
                       ['Male terms', 'Female terms'], ['Career', 'Family'])
gender_query_2 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']],
                       [WEAT_wordsets['science'], WEAT_wordsets['arts']],
                       ['Male terms', 'Female terms'], ['Science', 'Arts'])
gender_query_3 = Query([WEAT_wordsets['male_terms'], WEAT_wordsets['female_terms']],
                       [WEAT_wordsets['math'], WEAT_wordsets['arts_2']],
                       ['Male terms', 'Female terms'], ['Math', 'Arts'])

gender_query_4 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
    RND_wordsets['adjectives_intelligence'],
    RND_wordsets['adjectives_appearance']
], ['Male terms', 'Female terms'], ['Intelligence', 'Appearence'])

gender_query_5 = Query([RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
    RND_wordsets['adjectives_intelligence'],
    RND_wordsets['adjectives_sensitive']
], ['Male terms', 'Female terms'], ['Intelligence', 'Sensitive'])


gender_sent_1 = Query(
    [RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
        sentiments_wordsets['positive_words'],
        sentiments_wordsets['negative_words']
    ], ['Male terms', 'Female terms'], ['Positive words', 'Negative words'])

gender_role_1 = Query(
    [RND_wordsets['male_terms'], RND_wordsets['female_terms']], [
        debias_multiclass_wordsets['male_roles'],
        debias_multiclass_wordsets['female_roles']
    ], ['Male terms', 'Female terms'], ['Man Roles', 'Woman Roles'])

gender_queries = [gender_query_1, gender_query_2, gender_query_3, 
                  gender_query_4, gender_query_5, gender_role_1, gender_sent_1]

In [16]:
# Run the queries WEAT
WEAT_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    metric_params={'preprocessor_args': {
        'lowercase': True
    }},
    aggregate_results=True,
    return_only_aggregation=True,
    queries_set_name='Gender Queries')


# Run the queries using RNSB
RNSB_gender_results = run_queries(RNSB,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                return_only_aggregation=True,
                                queries_set_name='Gender Queries')




ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence and Appearence' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence and Sensitive' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Positive words and Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence and Appearence' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence an

In [17]:
WEAT_gender_results

,WEAT: Gender Queries average of abs values score
model_name,
u_desip,0.107393
p_desip,0.107472
sswe,0.293374
glove_cc,0.682374
glove_cc_large,0.841814


In [18]:
RNSB_gender_results

,RNSB: Gender Queries average of abs values score
model_name,
u_desip,0.045428
p_desip,0.042097
sswe,0.126451
glove_cc,0.067636
glove_cc_large,0.063590


In [19]:
# Run the queries using RND
RND_gender_results = run_queries(RND,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                generate_subqueries=True,
                                 return_only_aggregation=True,
                                queries_set_name='Gender Queries')




ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Positive words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


In [20]:
RND_gender_results

,RND: Gender Queries average of abs values score
model_name,
u_desip,0.026729
p_desip,0.026293
sswe,0.119304
glove_cc,0.140001
glove_cc_large,0.182953


In [21]:
# Run the queries using ECT
ECT_gender_results = run_queries(ECT,
                                gender_queries,
                                models,
                                metric_params={'preprocessor_args': {
                                    'lowercase': True
                                }},
                                aggregate_results=True,
                                 return_only_aggregation=True,
                                generate_subqueries=True,
                                queries_set_name='Gender Queries')

ERROR:root:At least one set of 'Male terms and Female terms wrt Intelligence' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Positive words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.
ERROR:root:At least one set of 'Male terms and Female terms wrt Negative words' query has proportionally fewer embeddings than allowed by the lost_vocabulary_threshold parameter (0.2). This query will return np.nan.


In [22]:
ECT_gender_results

,ECT: Gender Queries average of abs values score
model_name,
u_desip,0.940929
p_desip,0.931339
sswe,0.931320
glove_cc,0.938996
glove_cc_large,0.868008
